Demonstrate generation and readback of multiple tones.  Use connections as described in demo_01_one_simulated_resonator.ipynb

In [ ]:
import sys
sys.path.append('../../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
board = Scan.getBoard()
if board == 'zcu111':
    bitfile = 'mkids_2x2_kidsim_v1'    
elif board == 'zcu216':
    bitfile = 'mkids_2x2_kidsim_v2'
elif board == 'rfsoc4x2':
    bitfile = 'mkids_v3'

else:
    raise Exception("Do not know what to do with %s"%board)
scan = Scan.Scan(bitfile, iKids=0, iSimu=0)

In [ ]:
fMixer = 512
fTone = 814
decimation = 2
g = 0.9


In [ ]:
# Initialize simulation chain
simu = scan.simuChain
simu.analysis.qout(3)
simu.synthesis.qout(3)
simu.alloff()

# Get the kids chain
kids = scan.kidsChain

# Set the mixer of the analysis and synthesis chains to be the same

fMixerQ = kids.fq(fMixer)
_ = scan.set_mixer(fMixerQ)

In [ ]:
pfb = getattr(kids.soc, kids.synthesis.dict['chain']['pfb'])
N = pfb.dict['N']
chsel = getattr(kids.soc, kids.analysis.dict['chain']['chsel'])


In [ ]:
ntrans = np.zeros(N)
idxs = np.zeros(N)
for ch in range(N):
    ntrans[ch], addr, bit = chsel.ch2tran(ch)
    idxs[ch] = chsel.ch2idx(ch)
plt.plot(ntrans,'+',label="ntrans")
plt.plot(idxs,'x',label='idxs')
plt.xlabel("Input Channel Number")
plt.ylabel("Value")
plt.title(bitfile)
plt.legend()
plt.show()
plt.plot(ntrans[:100],'+',label="ntrans")
plt.plot(idxs[:100],'x',label='idxs')
plt.xlabel("Input Channel Number")
plt.ylabel("Value")
plt.legend()
plt.title(bitfile)


In [ ]:
qout = 2
kids.analysis.qout(qout)
kids.synthesis.qout(qout)
freqs = np.array([fTone+22.34, fTone])
fis = 0.123*np.arange(len(freqs))
gs = g*np.ones(len(freqs))/len(freqs)
gs[0] /= 2
cgs = None
print(gs)
kids.set_tones(freqs,fis,gs,cgs, verbose=True)

In [ ]:
print("here are the values of ntrans and idxs that will be read")
for i, (ntran,idx) in enumerate(zip(kids.ntrans,kids.idxs)):
    print("iTone=%2d   ntrans=%3d    idxs=%2d"%(i,ntran,idx))

In [ ]:
# This is the analysis chain, useful if you want to see what is going on
kids.analysis

In [ ]:
# This is the streamer
streamer_b = getattr(scan.soc, kids.analysis.dict['chain']['streamer'])
print(streamer_b)

In [ ]:
# The drivers check that this is true to see if there is anything to read
kids.analysis.anyenabled()

In [ ]:
# Note that MORE channels than requested show up here, because they are enabled in chunks of 8
chsel = getattr(scan.soc, kids.analysis.dict['chain']['chsel'])
print(type(chsel))
print("Here are the enabled channels for chsel.enabled_channels:", chsel.enabled_channels)


In [ ]:
xs = kids.get_xs() # The first read might have "old" data in the first few samples


In [ ]:
xs = kids.get_xs(verbose=True) # This data will be clean. verbose=2 shows progress
print("Returned data in the list xs")
for i,x in enumerate(xs):
    print("tone number %d has %d samples"%(i, len(x)))

In [ ]:
phi0 = np.angle(xs[0])
phi1 = np.angle(xs[1])
plt.plot(phi0, label="Tone 0")
plt.plot(phi1, label="Tone 1")
plt.xlabel("sample number")
plt.ylabel("phase (Radians)")
plt.legend()

In [ ]:
amp0 = np.abs(xs[0])
amp1 = np.abs(xs[1])
plt.plot(amp0, label="Tone 0")
plt.plot(amp1, label="Tone 1")
plt.xlabel("sample number")
plt.ylabel("amplitude")
plt.legend()